# Power electricity power consumption prediction model

Let's first start by importing the needed libraries

In [1]:
from pyspark.sql import SparkSession

And create a local parallel spark session

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Power electricity prediction") \
        .getOrCreate()

The we read the dataframe we need to apply the prediction on

In [3]:
train = spark.read.csv('../data/engineered/powerelectricity_train.csv', header=True, inferSchema=True)
test = spark.read.csv('../data/engineered/powerelectricity_test.csv', header=True, inferSchema=True)

Now we will test some regression models and evaluate them using rmse

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [5]:
train_ep = train.drop('rms_current')
train_ep = train.drop('electric_power')

#train_rms = train.drop('electric_power')

test_ep = test.drop('rms_current')
test_ep = test.drop('electric_power')

#test_rms = test.drop('electric_power')

features_ep = train_ep.columns.copy()
#features_ep.remove('electric_power')
features_ep.remove('delta_y')

#features_rms = train_rms.columns.copy()
#features_rms.remove('rms_current')

In [6]:
assembler_pe_train = VectorAssembler(inputCols=features_ep,outputCol="features")
train_ep = assembler_pe_train.transform(train_ep)

Let's start by predicting electric_power

In [7]:
rf_pe = RandomForestRegressor(featuresCol="features",labelCol='delta_y',numTrees=40,maxDepth=8,seed=42)
#rf_pe = GBTRegressor(featuresCol="features",labelCol='electric_power',seed=42)
#rf_pe = LinearRegression(featuresCol="features",labelCol='electric_power',maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [8]:
%%time

model_pe = rf_pe.fit(train_ep)

CPU times: user 23.8 ms, sys: 50 µs, total: 23.9 ms
Wall time: 32.1 s


In [9]:
assembler_pe_test = VectorAssembler(inputCols=features_ep,outputCol="features")
test_ep = assembler_pe_test.transform(test_ep)

In [10]:
predictions = model_pe.transform(test_ep)

In [11]:
evaluator_rmse = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R squared (r2) on test data = %g" % r2)

evaluator_mae = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Average Error (RMSE) on test data = %g" % mae)

Root Mean Squared Error (RMSE) on test data = 0.736133
R squared (r2) on test data = 0.938028
Mean Average Error (RMSE) on test data = 0.536951


In [12]:
predictions.select(["delta_y", "prediction"]).show(1000)

+--------------------+-------------------+
|             delta_y|         prediction|
+--------------------+-------------------+
|  2.4217561824393616| 2.2014194813700034|
|  2.4217561824393616|  2.085008478722529|
|  2.4217561824393616|  2.096602248830222|
|  2.4217561824393616| 2.1196234309647655|
|  2.4217561824393616| 2.1474913894216225|
|  2.4217561824393616|  2.130105551533508|
|  2.4217561824393616| 2.1297185903288693|
|  2.4217561824393616|  2.100577182499627|
|  2.4217561824393616| 2.1820006341890945|
|  2.4217561824393616|  2.204261423914818|
|  2.4217561824393616| 2.1924827547578367|
|  2.4217561824393616|  2.130105551533508|
|  2.4217561824393616| 2.1045929109410695|
|  2.4217561824393616|  2.166970114165398|
|  2.4217561824393616| 2.2042641806773453|
|  2.4217561824393616| 2.1993837153146822|
|  2.4217561824393616|  2.221647261802933|
|  2.4217561824393616|  2.204261423914818|
|  2.4217561824393616|  2.204261423914818|
|  2.4217561824393616| 2.1418842206904896|
|  2.421756